# DS Project no. 2: Kickstarter data set  
by Sandra Schwegmann & Marvin Lorenzen

## Project Goal

## Data Describtion

# Data Handling

In [596]:
import pandas as pd
import glob
import numpy as np
import operator

In [597]:
# Read in data: loop over all files
path = '/Users/sandra/Documents/materials/Project-2_Kickstarter/data'
filenames = glob.glob(path + "/*.csv")
comlete_csv =[]

for i in filenames:
    comlete_csv.append(pd.read_csv(i,encoding='utf-8'))

df = pd.concat(comlete_csv, ignore_index = True)


In [598]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209222 entries, 0 to 209221
Data columns (total 37 columns):
backers_count               209222 non-null int64
blurb                       209214 non-null object
category                    209222 non-null object
converted_pledged_amount    209222 non-null int64
country                     209222 non-null object
created_at                  209222 non-null int64
creator                     209222 non-null object
currency                    209222 non-null object
currency_symbol             209222 non-null object
currency_trailing_code      209222 non-null bool
current_currency            209222 non-null object
deadline                    209222 non-null int64
disable_communication       209222 non-null bool
friends                     300 non-null object
fx_rate                     209222 non-null float64
goal                        209222 non-null float64
id                          209222 non-null int64
is_backing                  300 

In [599]:
df.head(2)

,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,315,Babalus Shoes,"{""id"":266,""name"":""Footwear"",""slug"":""fashion/fo...",28645,US,1541459205,"{""id"":2094277840,""name"":""Lucy Conroy"",""slug"":""...",USD,$,True,...,babalus-childrens-shoes,https://www.kickstarter.com/discover/categorie...,False,False,live,1548223375,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",28645.0,international
1,47,A colorful Dia de los Muertos themed oracle de...,"{""id"":273,""name"":""Playing Cards"",""slug"":""games...",1950,US,1501684093,"{""id"":723886115,""name"":""Lisa Vollrath"",""slug"":...",USD,$,True,...,the-ofrenda-oracle-deck,https://www.kickstarter.com/discover/categorie...,True,False,successful,1504976459,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",1950.0,domestic


* data like dictonaries: 
    * category
    * creator
    * location
    * photo
    * profile
* photo will be dropped, other stay at the moment
* other object variables:
    * friends    - nothing in friends - dropped
    * is_backing - False
    * is_starred - False
    * permission - nothing in friends - dropped
* also drop source_url and urls

In [600]:
#drop first columns
df.drop(['photo','friends','permissions','currency_symbol',\
             'source_url','urls'],axis=1, inplace=True)

In [601]:
#df.location[df.location.isnull()!=False] = {}

In [602]:
# clean data from empty spaces, line breaks, etc.
dict_list1 = ['location','profile','creator']
for names in dict_list1:
    df[names] = df[names].str.replace(':null,',':None,')
    df[names] = df[names].str.replace(':false,',':False,')
    df[names] = df[names].str.replace(':true',':True')
    df[names] = df[names].str.replace('\n', '')
    df[names] = df[names].str.replace('\t', '')
    

In [603]:
# get fom creator ids -> remaining parts are ignored
# First separate creator elements
new2 = df["creator"].str.replace(", ", "_").str.split(",", n = 8, expand = True)
# now pick first column in new2 and split at :
test = new2[0].str.split(":")
# pick 2. column in test (where ids are written in)
test2 = list(map(operator.itemgetter(1), test))
# and make a new column in df with ids
df["creator_id"]=pd.Series(test2)
# convert ids to numeric values
df["creator_id"] = pd.to_numeric(df["creator_id"])

* We excluded everything after blurb as we could not manage to remove wrong entries -> has to be fixed...

In [604]:
# prepare profile data
# First separate creator elements
#new  = df["profile"].str.replace(", ", "_")
new  = df["profile"].str.replace(',"', ';;"')
new2 = new.str.split(";;", n = 16, expand = True)                                 
#new2 = new.str.replace(",000 ", "000").str.split(",", n = 16, expand = True)
# now drop column 4:
new3 = new2.drop(4, axis =1)
# change blurb to has or has no blurb
new3[5][new3[5]!='"blurb":"none"'] = '"blurb":"1"' #0 if new3[5]== "none" else 1
new3[5][new3[5]=='"blurb":"none"'] = '"blurb":"0"' #0 if new3[5]== "none" else 1

# change link_text to has or has not
#new3[10][new3[10]!='"link_text":"none"'] = '"link_text":"1"' #0 if new3[5]== "none" else 1
#new3[10][new3[10]=='"link_text":"none"'] = '"link_text":"0"' #0 if new3[5]== "none" else 1

new3[17]='"dummy":"dummy"}'
# pick columns of interest: 
#new3["profile"]= new3[[0,1,2,3,5,6,7,8,9,10,11,12,13,17]].apply(lambda x: ','.join(x), axis=1)
new3["profile"]= new3[[0,1,2,3,5,17]].apply(lambda x: ','.join(x), axis=1)
# write data in df
df["profile_test"] =new3["profile"]

In [605]:
# prepare location data
# dummy for NaN entries
location_dummy= '{"id":"","name":"","slug":"","short_name":"","displayable_name":"","localized_name":"",\
                "country":"","state":"","type":"","is_root":None,"urls":{}}'
#replace NaNs with dummy
df.location[df.location.isnull()] = location_dummy

/Users/sandra/opt/anaconda3/envs/nf/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [606]:
# extraction of data dictonaries in 'category','location'
dict_list = ['category','location','profile_test']#['category','location','profile''creator']#, null false true

for names in dict_list:
    print(names)
    new_name     = names+'new'
    suffix       = names+'_'
    df[new_name] = df[names].apply(eval)
    test         = df[new_name].apply(pd.Series)
    test         = test.add_prefix(suffix)
    df           = pd.concat([df,test],axis=1)  
 

category
location
profile_test


## General Overview

In [607]:
df.columns

Index(['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'created_at', 'creator', 'currency',
       'currency_trailing_code', 'current_currency', 'deadline',
       'disable_communication', 'fx_rate', 'goal', 'id', 'is_backing',
       'is_starrable', 'is_starred', 'launched_at', 'location', 'name',
       'pledged', 'profile', 'slug', 'spotlight', 'staff_pick', 'state',
       'state_changed_at', 'static_usd_rate', 'usd_pledged', 'usd_type',
       'creator_id', 'profile_test', 'categorynew', 'category_id',
       'category_name', 'category_slug', 'category_position',
       'category_parent_id', 'category_color', 'category_urls', 'locationnew',
       'location_id', 'location_name', 'location_slug', 'location_short_name',
       'location_displayable_name', 'location_localized_name',
       'location_country', 'location_state', 'location_type',
       'location_is_root', 'location_urls', 'profile_testnew',
       'profile_test_id', 'profile_test_pro

In [609]:
df2 = df
df2.drop(['category','location','locationnew','profile','profile_test','profile_testnew','creator','profile_test_dummy'], axis=1,inplace=True)

KeyError: "['category' 'location' 'locationnew' 'profile' 'profile_test' 'creator'\n 'profile_test_dummy'] not found in axis"

In [610]:
df2.head()


,backers_count,blurb,converted_pledged_amount,country,created_at,currency,currency_trailing_code,current_currency,deadline,disable_communication,...,location_state,location_type,location_is_root,location_urls,profile_testnew,profile_test_id,profile_test_project_id,profile_test_state,profile_test_state_changed_at,profile_test_blurb
0,315,Babalus Shoes,28645,US,1541459205,USD,True,USD,1552539775,False,...,CA,Town,False,{'web': {'discover': 'https://www.kickstarter....,"{'id': 3508024, 'project_id': 3508024, 'state'...",3508024,3508024,inactive,1541459205,1
1,47,A colorful Dia de los Muertos themed oracle de...,1950,US,1501684093,USD,True,USD,1504976459,False,...,TX,Town,False,{'web': {'discover': 'https://www.kickstarter....,"{'id': 3094785, 'project_id': 3094785, 'state'...",3094785,3094785,active,1505402530,1
2,271,"Electra's long awaited, eclectic Debut Pop/Roc...",22404,US,1348987533,USD,True,USD,1371013395,False,...,CA,Suburb,False,{'web': {'discover': 'https://www.kickstarter....,"{'id': 359847, 'project_id': 359847, 'state': ...",359847,359847,inactive,1425915819,1
3,3,The Mist of Tribunal is a turn-based card game...,165,GB,1483780271,GBP,False,USD,1489425776,False,...,Kaunas County,Town,False,{'web': {'discover': 'https://www.kickstarter....,"{'id': 2825329, 'project_id': 2825329, 'state'...",2825329,2825329,inactive,1483780271,1
4,3,"Livng with a brain impairment, what its like t...",2820,US,1354817071,USD,True,USD,1357763527,False,...,MI,Town,False,{'web': {'discover': 'https://www.kickstarter....,"{'id': 417385, 'project_id': 417385, 'state': ...",417385,417385,inactive,1425915823,1


In [612]:
df.is_backing[df.is_backing.isnull()==False] 

27        False
93        False
3233      False
3540      False
3597      False
4337      False
4382      False
6768      False
6776      False
6855      False
6923      False
6969      False
7117      False
7241      False
7292      False
7350      False
7376      False
7384      False
8614      False
8696      False
8838      False
11578     False
11988     False
12761     False
15222     False
15446     False
16230     False
16271     False
16275     False
16457     False
          ...  
188608    False
188676    False
188931    False
189070    False
189336    False
189448    False
189532    False
189754    False
189872    False
190795    False
191029    False
191156    False
191382    False
191522    False
191597    False
192037    False
195198    False
195224    False
198150    False
200699    False
200922    False
202738    False
202771    False
202838    False
202975    False
203030    False
203171    False
203342    False
204855    False
205011    False
Name: is_backing, Length

## Missing data

In [584]:
missing = pd.DataFrame(df.isnull().sum(),columns=['Number'])
missing[missing.Number!=0]

,Number
blurb,8
is_backing,208922
is_starred,208922
usd_type,480
category_parent_id,9041
location_slug,497
location_state,63
location_is_root,226


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209222 entries, 0 to 209221
Data columns (total 60 columns):
backers_count                    209222 non-null int64
blurb                            209214 non-null object
category                         209222 non-null object
converted_pledged_amount         209222 non-null int64
country                          209222 non-null object
created_at                       209222 non-null int64
creator                          209222 non-null object
currency                         209222 non-null object
currency_trailing_code           209222 non-null bool
current_currency                 209222 non-null object
deadline                         209222 non-null int64
disable_communication            209222 non-null bool
fx_rate                          209222 non-null float64
goal                             209222 non-null float64
id                               209222 non-null int64
is_backing                       300 non-null object
is_

In [ ]:
new2[new2[6].str.contains("she was asking for it")]
#df.profile[119063]